# Análise de Dados: Resultados Históricos do Futebol Internacional

**Autor:** Thomaz Colalillo Navajas 364869

**Disciplina:** Distributed Data Processing & Storage

---

Este notebook realiza uma análise exploratória sobre um dataset de resultados de partidas de futebol entre seleções, utilizando PySpark no ambiente Databricks.


## Etapa 1: Configuração e Carregamento dos Dados

Nesta primeira etapa, realizamos as seguintes ações:
1.  **Importamos as bibliotecas** necessárias para a análise (`requests`, `pyspark.sql.functions`, etc.).
2.  **Baixamos o arquivo CSV** diretamente de um repositório público no GitHub. Isso garante que o código seja reprodutível sem a necessidade de uploads manuais.
3.  **Carregamos os dados** em um DataFrame Spark, utilizando a biblioteca Pandas como uma ponte para facilitar a leitura.
4.  **Limpamos e renomeamos as colunas** para nomes mais intuitivos em português e garantimos que os tipos de dados (especialmente os placares) estejam corretos para cálculos.
5.  **Armazenamos o DataFrame em cache** (`.cache()`) para otimizar o desempenho das consultas que serão feitas a seguir.

In [0]:
# Importando as funções e bibliotecas necessárias
import requests
from io import StringIO
import pandas as pd
from pyspark.sql.functions import col, sum, abs, desc

url_raw_csv = "https://raw.githubusercontent.com/NavajasThomaz/Distributed_data_analysis_with_Databricks/refs/heads/main/results.csv" 

# Baixando o conteúdo do arquivo
try:
  response = requests.get(url_raw_csv)
  response.raise_for_status()
  csv_data_string = response.text
  print("Arquivo CSV baixado com sucesso do GitHub!")
except requests.exceptions.RequestException as e:
  print(f"Erro ao baixar o arquivo: {e}")
  dbutils.notebook.exit("Falha ao carregar os dados. Verifique a URL.")

# Usando Pandas como ponte para ler a string e criar um DataFrame Spark
csv_file_like_object = StringIO(csv_data_string)
pandas_df = pd.read_csv(csv_file_like_object)
df_original = spark.createDataFrame(pandas_df)

# Limpando e renomeando as colunas para facilitar a análise
df = df_original.withColumnRenamed("date", "data") \
                .withColumnRenamed("home_teamName", "time_mandante") \
                .withColumnRenamed("away_teamName", "time_visitante") \
                .withColumnRenamed("home_scoreHome", "placar_mandante") \
                .withColumnRenamed("away_scoreAway", "placar_visitante") \
                .withColumnRenamed("tournamentName", "torneio") \
                .withColumnRenamed("cityCity", "cidade") \
                .withColumnRenamed("countryCountry", "pais") \
                .withColumnRenamed("neutralTRUE", "campo_neutro")

# Garantindo que os placares são do tipo inteiro para cálculos
df = df.withColumn("placar_mandante", col("placar_mandante").cast("integer"))
df = df.withColumn("placar_visitante", col("placar_visitante").cast("integer"))

print("\nSchema do DataFrame após limpeza:")
df.printSchema()

Arquivo CSV baixado com sucesso do GitHub!

Schema do DataFrame após limpeza:
root
 |-- data: string (nullable = true)
 |-- time_mandante: string (nullable = true)
 |-- time_visitante: string (nullable = true)
 |-- placar_mandante: integer (nullable = true)
 |-- placar_visitante: integer (nullable = true)
 |-- torneio: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- campo_neutro: boolean (nullable = true)



## Etapa 2: Análise Exploratória dos Dados

Agora, vamos responder às perguntas solicitadas para extrair insights do dataset.

### 1 - Quantos registros existem na base?

In [0]:
total_registros = df.count()
print(f"Resposta: Existem {total_registros} registros (partidas) na base de dados.")

Resposta: Existem 40839 registros (partidas) na base de dados.


### 2 - Quantas equipes únicas mandantes existem na base?

In [0]:
equipes_mandantes_unicas = df.select("time_mandante").distinct().count()
print(f"Resposta: Existem {equipes_mandantes_unicas} equipes únicas que jogaram como mandantes.")

Resposta: Existem 309 equipes únicas que jogaram como mandantes.


### 3 - Quantas vezes as equipes mandantes saíram vitoriosas?

In [0]:
vitorias_mandantes = df.filter(col("placar_mandante") > col("placar_visitante")).count()
print(f"Resposta: As equipes mandantes venceram {vitorias_mandantes} vezes.")

Resposta: As equipes mandantes venceram 19864 vezes.


### 4 - Quantas vezes as equipes visitantes saíram vitoriosas?

In [0]:
vitorias_visitantes = df.filter(col("placar_visitante") > col("placar_mandante")).count()
print(f"Resposta: As equipes visitantes venceram {vitorias_visitantes} vezes.")

Resposta: As equipes visitantes venceram 11544 vezes.


### 5 - Quantas partidas resultaram em empate?

In [0]:
empates = df.filter(col("placar_mandante") == col("placar_visitante")).count()
print(f"Resposta: {empates} partidas terminaram em empate.")

Resposta: 9431 partidas terminaram em empate.


### 6 - Quantas partidas foram realizadas em cada país?

In [0]:
partidas_por_pais = df.groupBy("pais").count().orderBy(desc("count"))
print("Resposta: A tabela abaixo mostra a contagem de partidas por país, em ordem decrescente.")
partidas_por_pais.show()

Resposta: A tabela abaixo mostra a contagem de partidas por país, em ordem decrescente.
+--------------------+-----+
|                pais|count|
+--------------------+-----+
|       United States| 1144|
|              France|  801|
|             England|  687|
|            Malaysia|  644|
|              Sweden|  637|
|             Germany|  581|
|              Brazil|  529|
|               Spain|  517|
|            Thailand|  483|
|               Italy|  480|
|         Switzerland|  477|
|             Austria|  475|
|United Arab Emirates|  472|
|        South Africa|  470|
|               Qatar|  467|
|         South Korea|  453|
|           Argentina|  449|
|             Hungary|  431|
|               Chile|  405|
|             Belgium|  396|
+--------------------+-----+
only showing top 20 rows


### 7 - Qual país teve mais partidas?

In [0]:
# Reutilizamos o resultado da célula anterior e pegamos o primeiro registro
pais_com_mais_partidas = partidas_por_pais.first()
nome_pais = pais_com_mais_partidas["pais"]
num_partidas = pais_com_mais_partidas["count"]
print(f"Resposta: O país que sediou mais partidas foi {nome_pais}, com um total de {num_partidas} jogos.")

Resposta: O país que sediou mais partidas foi United States, com um total de 1144 jogos.


### 8 - Qual a partida com maior número de gols?

In [0]:
partida_mais_gols = df.withColumn("total_gols", col("placar_mandante") + col("placar_visitante")) \
                      .orderBy(desc("total_gols")) \
                      .first()

mandante = partida_mais_gols["time_mandante"]
visitante = partida_mais_gols["time_visitante"]
placar_m = partida_mais_gols["placar_mandante"]
placar_v = partida_mais_gols["placar_visitante"]
total_gols = partida_mais_gols["total_gols"]
data = partida_mais_gols["data"]

print(f"Resposta: A partida com maior número de gols ({total_gols} gols) foi {mandante} {placar_m} x {placar_v} {visitante}, em {data}.")

Resposta: A partida com maior número de gols (31 gols) foi Australia 31 x 0 American Samoa, em 2001-04-11.


### 9 - Qual a maior goleada?
(Definida pela maior diferença de gols entre o vencedor e o perdedor)

In [0]:
maior_goleada = df.withColumn("diferenca_gols", abs(col("placar_mandante") - col("placar_visitante"))) \
                   .orderBy(desc("diferenca_gols")) \
                   .first()

mandante = maior_goleada["time_mandante"]
visitante = maior_goleada["time_visitante"]
placar_m = maior_goleada["placar_mandante"]
placar_v = maior_goleada["placar_visitante"]
diferenca = maior_goleada["diferenca_gols"]
data = maior_goleada["data"]

print(f"Resposta: A maior goleada foi de {diferenca} gols de diferença, na partida {mandante} {placar_m} x {placar_v} {visitante}, em {data}.")

Resposta: A maior goleada foi de 31 gols de diferença, na partida Australia 31 x 0 American Samoa, em 2001-04-11.


### 10 - Quantos jogos ocorreram no Brasil?

In [0]:
jogos_no_brasil = df.filter(col("pais") == "Brazil").count()
print(f"Resposta: Ocorreram {jogos_no_brasil} jogos no Brasil.")

Resposta: Ocorreram 529 jogos no Brasil.
